In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Kombination vo Fehlerminderungsoptionen mit dem Estimator-Primitive

*Nutzungsschätzung: Sieben Minuten auf an Heron r2-Prozessor (HINWEIS: Des is nur a Schätzung. Ihri Laufzeit kann variieren.)*

## Hintergrund
Dä Walkthrough schaut sich de Fehlerunterdrückungs- und Fehlerminderungsoptionen an, de beim Estimator-Primitive vo Qiskit Runtime zur Verfügung stehng. Sie wern a Schaltung und a Observable konstruiern und Jobs mit dem Estimator-Primitive unter Verwendung vo verschiedene Kombinationen vo Fehlerminderungseinstellungen einreichen. Danach zeichnts Sie de Ergebnisse auf, um de Auswirkungen vo de verschiedene Einstellungen z'beobachten. De meisten Beispiele verwendn a 10-Qubit-Schaltung, um Visualisierungen z'erleichtern, und am End kenna Sie den Workflow auf 50 Qubits skalieren.

Des san de Fehlerunterdrückungs- und Minderungsoptionen, de Sie verwendn wern:

- Dynamical Decoupling
- Messfehlerkompensation
- Gate Twirling
- Zero-Noise Extrapolation (ZNE)

## Anforderungen
Schaugn Sie drauf, dass Sie vor dem Beginn vo dem Walkthrough Folgendes installiert ham:

- Qiskit SDK v2.1 oder höher, mit Unterstützung für [Visualisierung](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.40 oder höher (`pip install qiskit-ibm-runtime`)

## Setup

In [7]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit.circuit.library import efficient_su2, unitary_overlap
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Batch, EstimatorV2 as Estimator

## Schritt 1: Klassische Eingaben auf a Quantenproblem abbilden
Dä Walkthrough geht davon aus, dass des klassische Problem scho auf Quantenmechanik abgbildet worn is. Fangts Sie mit da Konstruktion vo a Schaltung und a Observable zum Messen aa. Weil de Techniken, de da verwendet wern, auf viele verschiedene Arten vo Schaltungen anwendbar san, verwendet dä Walkthrough dä Einfachheit halber de [`efficient_su2`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.efficient_su2)-Schaltung aus da Qiskit-Schaltungsbibliothek.

`efficient_su2` is a parametrisierte Quantenschaltung, de so konzipiert is, dass sie auf Quantenhardware mit begrenzter Qubit-Konnektivität effizient ausführbar is und dennoch ausdrucksstark gnua, um Probleme in Anwendungsdomänen wia Optimierung und Chemie z'lösen. Sie wird durch abwechselnde Schichten vo parametrisierte Ein-Qubit-Gates mit ana Schicht konstruiert, de a feschts Muster vo Zwei-Qubit-Gates enthält, für a gwählte Anzahl vo Wiederholungen. Des Muster vo de Zwei-Qubit-Gates kann vom Benutzer spezifiziert wern. Da kenna Sie des eingebaute `pairwise`-Muster verwenden, weil's de Schaltungstiefe minimiert, indem's de Zwei-Qubit-Gates so dicht wia möglich packt. Des Muster kann nur mit linearer Qubit-Konnektivität ausgeführt wern.

In [4]:
n_qubits = 10
reps = 1

circuit = efficient_su2(n_qubits, entanglement="pairwise", reps=reps)

circuit.decompose().draw("mpl", scale=0.7)

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/24abd7ba-bbb8-443b-9e81-866795d39a6c-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/24abd7ba-bbb8-443b-9e81-866795d39a6c-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/24abd7ba-bbb8-443b-9e81-866795d39a6c-0.avif)

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/24abd7ba-bbb8-443b-9e81-866795d39a6c-1.avif)

Für unsane Observable nehma mia de Pauli-$Z$-Operator, dea auf des letzte Qubit wirkt, $Z I \cdots I$.

In [5]:
# Z on the last qubit (index -1) with coefficient 1.0
observable = SparsePauliOp.from_sparse_list(
    [("Z", [-1], 1.0)], num_qubits=n_qubits
)

An dem Punkt kinnts Sie mit da Ausführung vo Ihrer Schaltung weitermachen und de Observable messen. Sie mechtn aber aa de Ausgabe vom Quantengerät mit da korrekten Antwort vergleichen – des hoaßt, dem theoretischen Wert vo da Observable, falls de Schaltung ohne Fehler ausgeführt worn wär. Für kloane Quantenschaltungen kenna Sie den Wert berechnen, indem Sie de Schaltung auf an klassischen Computer simulieren, aber des is für größere Utility-Scale-Schaltungen ned möglich. Sie kenna des Problem mit da "Spiegelschaltungs"-Technik (aa bekannt als "Compute-Uncompute") umgehen, de zum Benchmarking vo da Leistung vo Quantengeräten nützlich is.

#### Spiegelschaltung
Bei da Spiegelschaltungstechnik verkettn Sie de Schaltung mit ihrer inversen Schaltung, de durch Umkehrung vo jedem Gate da Schaltung in umgekehrter Reihenfolge gbildet wird. De resultierende Schaltung implementiert den Identitätsoperator, dea trivial simuliert wern kann. Weil de Struktur vo da ursprünglichen Schaltung in da Spiegelschaltung erhalten bleibt, gibt de Ausführung vo da Spiegelschaltung dennoch a Vorstellung davon, wia des Quantengerät bei da ursprünglichen Schaltung abschneid'n würd.

De folgende Codezelle weist Ihrer Schaltung zufällige Parameter zu und konstruiert dann de Spiegelschaltung unter Verwendung vo da [`unitary_overlap`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.unitary_overlap)-Klasse. Füagnts Sie vor dem Spiegeln vo da Schaltung a [Barrier](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.Barrier)-Instruktion ei, um z'verhindert, dass dä Transpiler de zwoa Teile vo da Schaltung auf boadn Seiten vo da Barrier zamführt. Ohne de Barrier würd dä Transpiler de ursprüngliche Schaltung mit ihrer Inversen zamführen, was zu ana transpilierten Schaltung ohne Gates führt.

In [8]:
# Generate random parameters
rng = np.random.default_rng(1234)
params = rng.uniform(-np.pi, np.pi, size=circuit.num_parameters)

# Assign the parameters to the circuit
assigned_circuit = circuit.assign_parameters(params)

# Add a barrier to prevent circuit optimization of mirrored operators
assigned_circuit.barrier()

# Construct mirror circuit
mirror_circuit = unitary_overlap(assigned_circuit, assigned_circuit)

mirror_circuit.decompose().draw("mpl", scale=0.7)

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/4dbde811-1ba9-47a8-85a0-dcaff054ed60-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/4dbde811-1ba9-47a8-85a0-dcaff054ed60-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/4dbde811-1ba9-47a8-85a0-dcaff054ed60-0.avif)

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/4dbde811-1ba9-47a8-85a0-dcaff054ed60-1.avif)

## Schritt 2: Problem für de Ausführung auf Quantenhardware optimieren
Sie müassn Ihre Schaltung optimieren, bevor Sie sie auf Hardware ausführen. Dä Prozess umfasst a paar Schritte:

- Wähln Sie a Qubit-Layout, des de virtuellen Qubits vo Ihrer Schaltung auf physische Qubits auf da Hardware abbildet.
- Füagnts nach Bedarf Swap-Gates ei, um Interaktionen zwischn Qubits z'routen, de ned verbunden san.
- Übersetzts Sie de Gates in Ihrer Schaltung in [Instruction Set Architecture (ISA)](/guides/transpile#instruction-set-architecture)-Instruktionen, de direkt auf da Hardware ausgeführt wern kenna.
- Führts Sie Schaltungsoptimierungen durch, um de Schaltungstiefe und Gate-Anzahl z'minimieren.

Dä in Qiskit eingebaute Transpiler kann all de Schritte für Sie durchführen. Weil des Beispiel a hardwareeffiziente Schaltung verwendet, soid dä Transpiler in da Lage sei, a Qubit-Layout z'wählen, des koa Swap-Gates zum Routing vo Interaktionen braucht.

Sie müassn des z'verwendende Hardwaregerät auswählen, bevor Sie Ihre Schaltung optimieren. De folgende Codezelle fordert des am wenigsten ausgelastete Gerät mit mindestens 127 Qubits aa.

In [9]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

Sie kenna Ihre Schaltung für Ihr gwähltes Backend transpiliern, indem Sie an Pass-Manager erstellen und dann den Pass-Manager auf da Schaltung ausführen. A einfache Möglichkeit, an Pass-Manager z'erstellen, is de Verwendung vo da Funktion [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.generate_preset_pass_manager). Schaugts Sie [Transpilierung mit Pass-Managern](/guides/transpile-with-pass-managers) für a detailliertere Erklärung vo da Transpilierung mit Pass-Managern.

In [10]:
pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=1234
)
isa_circuit = pass_manager.run(mirror_circuit)

isa_circuit.draw("mpl", idle_wires=False, scale=0.7, fold=-1)

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/027e829a-44d3-455e-b2bf-8ce0d7e26b9b-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/027e829a-44d3-455e-b2bf-8ce0d7e26b9b-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/027e829a-44d3-455e-b2bf-8ce0d7e26b9b-0.avif)

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/027e829a-44d3-455e-b2bf-8ce0d7e26b9b-1.avif)

De transpilierte Schaltung enthält jetzt nur no ISA-Instruktionen. De Ein-Qubit-Gates san in Bezug auf $\sqrt{X}$-Gates und $R_z$-Rotationen zerlegt worn, und de CX-Gates san in [ECR-Gates](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.ECRGate#ecrgate) und Ein-Qubit-Rotationen zerlegt worn.

Dä Transpilationsprozess hat de virtuellen Qubits vo da Schaltung auf physische Qubits auf da Hardware abgbildet. De Informationen über des Qubit-Layout san im `layout`-Attribut vo da transpilierten Schaltung gspreichert. De Observable is aa in Bezug auf de virtuellen Qubits definiert worn, daher müassn Sie des Layout auf de Observable anwenden, was Sie mit da Methode [`apply_layout`](https://docs.quantum.ibm.com/api/qiskit/qiskit.quantum_info.SparsePauliOp#apply_layout) vo `SparsePauliOp` toa kenna.

In [12]:
isa_observable = observable.apply_layout(isa_circuit.layout)

print("Original observable:")
print(observable)
print()
print("Observable with layout applied:")
print(isa_observable)

Original observable:
SparsePauliOp(['ZIIIIIIIII'],
              coeffs=[1.+0.j])

Observable with layout applied:
SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII'],
              coeffs=[1.+0.j])


## Step 3: Execute using Qiskit primitives

You are now ready to run your circuit using the Estimator primitive.

Here you will submit five separate jobs, starting with no error suppression or mitigation, and successively enabling various error suppression and mitigation options available in Qiskit Runtime. For information about the options, refer to the following pages:

- [Overview of all options](/docs/api/qiskit-ibm-runtime/options)
- [Dynamical decoupling](/docs/api/qiskit-ibm-runtime/options-dynamical-decoupling-options)
- [Resilience, including measurement error mitigation and zero-noise extrapolation (ZNE)](/docs/api/qiskit-ibm-runtime/options-resilience-options-v2)
- [Twirling](/docs/api/qiskit-ibm-runtime/options-twirling-options)

Because these jobs can run independently of each other, you can use [batch mode](/docs/guides/run-jobs-batch) to allow Qiskit Runtime to optimize the timing of their execution.

In [13]:
pub = (isa_circuit, isa_observable)

jobs = []

with Batch(backend=backend) as batch:
    estimator = Estimator(mode=batch)
    # Set number of shots
    estimator.options.default_shots = 100_000
    # Disable runtime compilation and error mitigation
    estimator.options.resilience_level = 0

    # Run job with no error mitigation
    job0 = estimator.run([pub])
    jobs.append(job0)

    # Add dynamical decoupling (DD)
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XpXm"
    job1 = estimator.run([pub])
    jobs.append(job1)

    # Add readout error mitigation (DD + TREX)
    estimator.options.resilience.measure_mitigation = True
    job2 = estimator.run([pub])
    jobs.append(job2)

    # Add gate twirling (DD + TREX + Gate Twirling)
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"
    job3 = estimator.run([pub])
    jobs.append(job3)

    # Add zero-noise extrapolation (DD + TREX + Gate Twirling + ZNE)
    estimator.options.resilience.zne_mitigation = True
    estimator.options.resilience.zne.noise_factors = (1, 3, 5)
    estimator.options.resilience.zne.extrapolator = ("exponential", "linear")
    job4 = estimator.run([pub])
    jobs.append(job4)

## Schritt 3: Ausführung mit Qiskit Primitives
Jetzt san Sie bereit, Ihre Schaltung mit dem Estimator-Primitive auszuführen.

Da reicnts Sie fünf separate Jobs ei, fangts ohne Fehlerunterdrückung oder -minderung aa, und aktivieren sukzessiv verschiedene Fehlerunterdrückungs- und -minderungsoptionen, de in Qiskit Runtime zur Verfügung stehng. Informationen zu de Optionen findn Sie auf de folgenden Seitn:

- [Übersicht über alle Optionen](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options)
- [Dynamical Decoupling](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-dynamical-decoupling-options)
- [Resilience, einschließlich Messfehlerkompensation und Zero-Noise Extrapolation (ZNE)](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-resilience-options-v2)
- [Twirling](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-twirling-options)

Weil de Jobs unabhängig voneinander ausgeführt wern kenna, kenna Sie den [Batch-Modus](/guides/run-jobs-batch) verwenden, damit Qiskit Runtime des Timing vo ihrer Ausführung optimieren kann.

In [14]:
# Retrieve the job results
results = [job.result() for job in jobs]

# Unpack the PUB results (there's only one PUB result in each job result)
pub_results = [result[0] for result in results]

# Unpack the expectation values and standard errors
expectation_vals = np.array(
    [float(pub_result.data.evs) for pub_result in pub_results]
)
standard_errors = np.array(
    [float(pub_result.data.stds) for pub_result in pub_results]
)

# Plot the expectation values
fig, ax = plt.subplots()
labels = ["No mitigation", "+ DD", "+ TREX", "+ Twirling", "+ ZNE"]
ax.bar(
    range(len(labels)),
    expectation_vals,
    yerr=standard_errors,
    label="experiment",
)
ax.axhline(y=1.0, color="gray", linestyle="--", label="ideal")
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels)
ax.set_ylabel("Expectation value")
ax.legend(loc="upper left")

plt.show()

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/eef38976-0ca2-429a-b2dc-41aac69605f7-0.avif" alt="Output of the previous code cell" />

## Schritt 4: Nachbearbeitung und Rückgabe vom Ergebnis im gwünschten klassischen Format
Jetzt kenna Sie de Daten analysieren. Da rufts Sie de Jobergebnisse ab, extrahieren de gmessenen Erwartungswerte aus ihnen und zeichnts de Werte auf, einschließlich Fehlerbalken vo ana Standardabweichung.

In [15]:
n_qubits = 50
reps = 1

# Construct circuit and observable
circuit = efficient_su2(n_qubits, entanglement="pairwise", reps=reps)
observable = SparsePauliOp.from_sparse_list(
    [("Z", [-1], 1.0)], num_qubits=n_qubits
)

# Assign parameters to circuit
params = rng.uniform(-np.pi, np.pi, size=circuit.num_parameters)
assigned_circuit = circuit.assign_parameters(params)
assigned_circuit.barrier()

# Construct mirror circuit
mirror_circuit = unitary_overlap(assigned_circuit, assigned_circuit)

# Transpile circuit and observable
isa_circuit = pass_manager.run(mirror_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

# Run jobs
pub = (isa_circuit, isa_observable)

jobs = []

with Batch(backend=backend) as batch:
    estimator = Estimator(mode=batch)
    # Set number of shots
    estimator.options.default_shots = 100_000
    # Disable runtime compilation and error mitigation
    estimator.options.resilience_level = 0

    # Run job with no error mitigation
    job0 = estimator.run([pub])
    jobs.append(job0)

    # Add dynamical decoupling (DD)
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XpXm"
    job1 = estimator.run([pub])
    jobs.append(job1)

    # Add readout error mitigation (DD + TREX)
    estimator.options.resilience.measure_mitigation = True
    job2 = estimator.run([pub])
    jobs.append(job2)

    # Add gate twirling (DD + TREX + Gate Twirling)
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"
    job3 = estimator.run([pub])
    jobs.append(job3)

    # Add zero-noise extrapolation (DD + TREX + Gate Twirling + ZNE)
    estimator.options.resilience.zne_mitigation = True
    estimator.options.resilience.zne.noise_factors = (1, 3, 5)
    estimator.options.resilience.zne.extrapolator = ("exponential", "linear")
    job4 = estimator.run([pub])
    jobs.append(job4)

# Retrieve the job results
results = [job.result() for job in jobs]

# Unpack the PUB results (there's only one PUB result in each job result)
pub_results = [result[0] for result in results]

# Unpack the expectation values and standard errors
expectation_vals = np.array(
    [float(pub_result.data.evs) for pub_result in pub_results]
)
standard_errors = np.array(
    [float(pub_result.data.stds) for pub_result in pub_results]
)

# Plot the expectation values
fig, ax = plt.subplots()
labels = ["No mitigation", "+ DD", "+ TREX", "+ Twirling", "+ ZNE"]
ax.bar(
    range(len(labels)),
    expectation_vals,
    yerr=standard_errors,
    label="experiment",
)
ax.axhline(y=1.0, color="gray", linestyle="--", label="ideal")
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels)
ax.set_ylabel("Expectation value")
ax.legend(loc="upper left")

plt.show()

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/d7d8408b-faf1-4eda-ab9c-bdeaab01ff53-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/eef38976-0ca2-429a-b2dc-41aac69605f7-0.avif)

In dem kleinen Maßstab is's schwierig, de Wirkung vo de meisten Fehlerminderungstechniken z'sehn, aber Zero-Noise Extrapolation bietet a spürbare Verbesserung. Aber schaugts, dass de Verbesserung ned umsonst kommt, weil des ZNE-Ergebnis aa an größern Fehlerbalken aufweist.

## Skalierung vom Experiment nach oben
Bei da Entwicklung vo an Experiment is's nützlich, mit ana kleinen Schaltung aazufangen, um Visualisierungen und Simulationen z'erleichtern. Nachdem Sie Ihren Workflow auf ana 10-Qubit-Schaltung entwickelt und getestet ham, kenna Sie ihn auf 50 Qubits skalieren. De folgende Codezelle wiederholt alle Schritte in dem Walkthrough, wendet sie aber jetzt auf a 50-Qubit-Schaltung aa.